# Topic 5: Sea-level Rise from Mass Gain and Thermal Expansion

In [ ]:
from cmda_utils import calc_anom, area_avg
import numpy as np
import cartopy
import cartopy.crs as ccrs
import xarray as xr ## xarray is a useful python package for analysis of geospatial data, read more here https://docs.xarray.dev/en/stable/
import matplotlib.pyplot as plt

In [ ]:
## EDIT LATER TO USE CMDA DATA LOADING TOOLS
## ALTIMETRY DATA
gmsl_altimetry_dectime = np.loadtxt('/home/jovyan/shared/data/gmsl/GMSL_TPJAOS_5.1_199209_202203.txt', skiprows=48)[:,2]
gmsl_altimetry_smoothed_sigrem = xr.open_dataset('/home/jovyan/shared/data/gmsl/GMSL_TPJAOS_5.1_199209_202203.nc').gmsl_GIA_applied_smoothed_sigrem # units = mm (anomaly)
gmsl_altimetry    = xr.open_dataset('/home/jovyan/shared/data/gmsl/GMSL_TPJAOS_5.1_199209_202203.nc').gmsl_GIA_applied # units = mm (anomaly)
## STERIC SEA LEVEL DATA
gmsl_steric = xr.open_dataset('/home/jovyan/shared/data/gmsl/noaa_gmssl.nc', decode_times=False).seas_s_mm_WO # units = mm (anomaly)
noaa_time = np.zeros(len(gmsl_steric.time), dtype='datetime64[s]')
for i in range(len(noaa_time)):
    noaa_time[i] = np.datetime64('1955-01-01 00:00:00', 'M')+np.timedelta64(np.int64(gmsl_steric.time[i]),'M')
gmsl_steric = gmsl_steric.assign_coords({'time':noaa_time})

In [ ]:
## GRACE DATA, takes longer than others to load...
grace_over_land_grid = calc_anom(xr.open_dataset('/home/jovyan/cmda_data/cmip5/others/nasa_grace_zl_a_200204-202105.nc').zl)
grace_over_ocean_grid = calc_anom(xr.open_dataset('/home/jovyan/cmda_data/cmip5/others/nasa_grace_zo_200204-202105.nc').zo)
grace_over_land = area_avg(grace_over_land_grid)
grace_over_ocean = area_avg(grace_over_ocean_grid)

### Question 1: Determine linear and quadratic (i.e. accelerating) trends in GMSL from MEaSUREs altimetry data

Note that this is a merged continuous 1993-present GMSL product from multiple altimetry missions (https://podaac.jpl.nasa.gov/dataset/MERGED_TP_J1_OSTM_OST_GMSL_ASCII_V51#). The timeseries here shows anomalies referenced to the 1996-2016 mean with seasonal signals removed and with a 60-day Gaussian type smoothing applied.

In [ ]:
## STUDENT EXERCISE ##
# linear fit...
# code should produce np.poly1d object, 'p_linear'

In [ ]:
# quadratic fits...
fit = np.polyfit(gmsl_altimetry_dectime-1993, gmsl_altimetry_smoothed_sigrem, 2)
print('Quadratic fit is [a t**2 + b t + c] where a = %4.3f, b = %4.3f, and c = %4.3f, (t is time in years)'%(fit[0],fit[1],fit[2]))
v = 2*fit[0]*(gmsl_altimetry_dectime-1993)+fit[1]
print('Mean rate (1992-2021): %4.3f mm/yr, Acceleration: %4.3f mm/yr2'%(np.mean(v), 2*fit[0]))
p_quadratic = np.poly1d(fit)

In [ ]:
fig,ax = plt.subplots(1,1, figsize=(10,7))
ax.plot(gmsl_altimetry_smoothed_sigrem.time, gmsl_altimetry_smoothed_sigrem, 'k-', linewidth=3, label='GMSL (smoothed with seasonal signal removed)')
ax.plot(gmsl_altimetry_smoothed_sigrem.time,p_quadratic(gmsl_altimetry_dectime-1993), 'b-', linewidth=1, label = 'Accelerating trend')
## uncomment once linear trend generated
# ax.plot(gmsl_altimetry_smoothed_sigrem.time,p_linear(gmsl_alitmetry_dectime-1993), 'b--', linewidth=1, label = 'Linear trend')
ax.legend(loc=0,fontsize=12)
ax.set_ylabel('MEaSUREs GMSL [mm]', fontsize=15)
ax.set_xlabel('Time', fontsize=15)
plt.show()

### Question 1 (cont.): If these trends are simply projected forward, how much would we anticipate sea level to rise between now and 2100? What issues are associated with such simple projections? What types of processes may be missed?

(calculate and discuss)

*hint: one way to approach this would be to construct an array of np.datetime64 dates running from 1993 to 2100 and inputing these arrays into p_linear and p_quadratic*

In [ ]:
## STUDENT EXERCISE ##

### Question 2: For the period where the records overlap, compare the contribution of mass change (from GRACE) and steric change (from NOAA WOA) to the total rise in GMSL. Why are the total, mass, and steric records available at such different temporal resolutions? Where do the records agree and differ? 

Our three datasets overlap for the period 2005-02 to 2021-02 so we want to just pull out this time period from all datasets. It's important that when we compare these anomalies they are all referenced to the same interval mean and use the same period for seasonal signal removal.

In [ ]:
# subset time period
gmsl_altimetry_overlap = gmsl_altimetry.sel(time=slice('2005-02','2021-02'))
gmsl_steric_overlap = gmsl_steric.sel(time=slice('2005-02','2021-02'))
grace_over_ocean_overlap = grace_over_ocean.sel(time=slice('2005-02','2021-02'))*10 # unit conversion
# remove climatological (seasonal mean) signal
gmsl_altimetry_overlap = gmsl_altimetry_overlap.groupby('time.month') - gmsl_altimetry_overlap.groupby('time.month').mean()
gmsl_steric_overlap = gmsl_steric_overlap.groupby('time.month') - gmsl_steric_overlap.groupby('time.month').mean()
grace_over_ocean_overlap = grace_over_ocean_overlap.groupby('time.month') - grace_over_ocean_overlap.groupby('time.month').mean()

In [ ]:
fig,ax = plt.subplots(1,1, figsize=(10,7))

ax.plot(gmsl_altimetry_overlap.time, gmsl_altimetry_overlap, 'k-', linewidth=3, label='Altimetry (total, 10 day)')
shift = grace_over_ocean_overlap.sel(time=slice('2005', '2006')).mean() - gmsl_altimetry_overlap.sel(time=slice('2005', '2006')).mean()
ax.plot(grace_over_ocean_overlap.time, grace_over_ocean_overlap-shift, 'b-', linewidth=2, label='GRACE (mass, monthly)')
shift = gmsl_steric_overlap.sel(time=slice('2005', '2006')).mean() - gmsl_altimetry_overlap.sel(time=slice('2005', '2006')).mean()
ax.plot(gmsl_steric_overlap.time, gmsl_steric_overlap-shift, 'r-', linewidth=2, label='NOAA (steric, 3 monthly)')

ax.legend(loc=0,fontsize=12)
ax.set_ylabel('GMSL [mm]', fontsize=15)
ax.set_xlabel('Time', fontsize=15)
plt.show()

### Question 2 (cont.): Do the steric and mass records add up to the total record? (Discuss)

(calculate and discuss)

*hint: you will need to either coarsen the total/mass products to the temporal resolution of the steric product (e.g. below) or interpolate the steric/mass products onto the time series of the total product*

In [ ]:
 ## STUDENT EXERCISE ##

### Question 2 (cont.): What are some possible reasons for the discrepancies between the NOAA derived steric sea level trend and that which can be obtained from the residual between altimetry and mass change?

In [ ]:
gmsl_avg = gmsl_altimetry_overlap.resample(time='1ME').mean('time')
steric_from_resid = gmsl_avg.sel(time=grace_over_ocean_overlap.time, method='nearest').values - grace_over_ocean_overlap.values

In [ ]:
fig,ax = plt.subplots(1,1, figsize=(10,7))

ax.plot(grace_over_ocean_overlap.time, steric_from_resid, 'k-', linewidth=2, label='Steric from residual of altimetry and mass (monthly)')
ax.plot(gmsl_steric_overlap.time, gmsl_steric_overlap, 'r-', linewidth=2, label='Steric from WOA (3 monthly)')

ax.legend(loc=0,fontsize=12)
ax.set_ylabel('GMSL [mm]', fontsize=15)
ax.set_xlabel('Time', fontsize=15)
plt.show()

### Question 3: The ocean is gaining mass from the land. Show this by comparing timeseries of grace_over_ocean and grace_over_land (discuss possible sources of difference). 

note: you will need to divide grace_over_land by 2 for them to agree, there seems to be a unit discrepancy in the database...

In [ ]:
## STUDENT EXERCISE ##

### Question 3 (cont.): Map changes in land water storage. Where is most of the mass loss from the land occurring? Do you see anything surprising?

This code just takes an anomaly between two epochs, you could also try computing linear trends in each grid box.

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature

In [ ]:
grace_over_land_first_decade = grace_over_land_grid.sel(time=slice('2002','2011')).mean('time')
grace_over_land_second_decade = grace_over_land_grid.sel(time=slice('2012','2021')).mean('time')

diff = grace_over_land_second_decade - grace_over_land_first_decade

In [ ]:
fig = plt.figure(figsize=[15, 7])
proj = ccrs.PlateCarree(central_longitude=180)
ax  = plt.subplot(projection = proj)
ax.add_feature(cartopy.feature.COASTLINE);
cf=ax.pcolormesh(diff.lon, diff.lat, diff, transform = ccrs.PlateCarree(), vmin=-150,vmax=150, cmap='RdBu')
cax = fig.add_axes([0.26, 0.05, 0.5, 0.03])
cbar=plt.colorbar(cf,cax = cax,orientation='horizontal',shrink = 0.5)
cax.set_xlabel('cm', fontsize = 15)
ax.set_title('Equivalent water height anomaly (2012-2021 mean - 2002-2011 mean)', fontsize=15)
plt.show()
